In [1]:
from keras.applications import VGG16
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fc8c2489ed0> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c89d0850> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c1e23450> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc8c15e5190> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c1e23f10> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c0588750> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc8c1e236d0> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c0597e90> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c059bd50> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c0530f10> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc8c248e0d0> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c0547e90> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c0558ed0> False
<keras.layers.convolutional.Conv2D object at 0x7fc8c04e9710> False
<keras.layers.pooling.MaxPooling2D object at 0x7fc8c0500bd0>

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
 
model.add(vgg_conv)
 
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(33, activation='softmax'))
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 33)                16929     
Total params: 40,947,553
Trainable params: 33,312,289
Non-trainable params: 7,635,264
__________________________________

In [5]:
train_dir = '/home/asus/Desktop/Dataset_splits/train'
validation_dir = '/home/asus/Desktop/Dataset_splits/validation'

In [6]:
import numpy as np
from random import shuffle

In [7]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
  height_shift_range=0.2,
    horizontal_flip=True,
   fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_batchsize = 124
val_batchsize = 20
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 22110 images belonging to 33 classes.
Found 4132 images belonging to 33 classes.


In [8]:
print(train_generator.class_indices)

{'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 0, 'Corn_(maize)___Common_rust_': 1, 'Corn_(maize)___healthy': 2, 'GD_Alternia': 3, 'GD_Deficiency': 4, 'GD_Healthy': 5, 'GD_Leaf_Blight': 6, 'GD_Rust': 7, 'GD_Spidermites': 8, 'GD_Toxicity': 9, 'GD_Webbloch': 10, 'Paddy_Bacterial_leaf_blight': 11, 'Paddy_Brown_spot': 12, 'Pepper__bell___Bacterial_spot': 13, 'Pepper__bell___healthy': 14, 'Potato___Early_blight': 15, 'Potato___Late_blight': 16, 'Potato___healthy': 17, 'Tomato_Bacterial_spot': 18, 'Tomato_Early_blight': 19, 'Tomato_Late_blight': 20, 'Tomato_Leaf_Mold': 21, 'Tomato_Septoria_leaf_spot': 22, 'Tomato_Spider_mites_Two_spotted_spider_mite': 23, 'Tomato__Target_Spot': 24, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 25, 'Tomato__Tomato_mosaic_virus': 26, 'Tomato_healthy': 27, 'pomo_Alternaria_Alternata': 28, 'pomo_Anthracnose': 29, 'pomo_Bacterial_Blight': 30, 'pomo_Cercospora_LeafSpot': 31, 'pomo_Healthy_Leaves': 32}


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/20
179/178 [==============================] - 4025s 22s/step - loss: 1.3475 - acc: 0.5898 - val_loss: 0.5141 - val_acc: 0.7580
Epoch 2/20
179/178 [==============================] - 4677s 26s/step - loss: 0.6417 - acc: 0.7890 - val_loss: 0.8464 - val_acc: 0.7289
Epoch 3/20
179/178 [==============================] - 4423s 25s/step - loss: 0.4833 - acc: 0.8436 - val_loss: 0.9738 - val_acc: 0.8785
Epoch 4/20
179/178 [==============================] - 4335s 24s/step - loss: 0.3777 - acc: 0.8726 - val_loss: 0.1003 - val_acc: 0.8703
Epoch 5/20
179/178 [==============================] - 4295s 24s/step - loss: 0.3322 - acc: 0.8916 - val_loss: 0.2424 - val_acc: 0.9054
Epoch 6/20
179/178 [==============================] - 4903s 27s/step - loss: 0.2898 - acc: 0.9041 - val_loss: 0.3179 - val_acc: 0.9027
Epoch 7/20
179/178 [==============================] - 4932s 28s/step - loss: 0.2545 - acc: 0.9137 - val_loss: 0.1803 - val_acc: 0.9027
Epoch 8/20
179/178 [==============================] - 4

In [10]:
model.save('finalized_groundnnnut_disease.h5')